In [1]:
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(25)

2025-03-17 15:44:38.529717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load data
csv_path = '../../datasets/human_or_ai_dataset_small.csv'  # Change this to your file path
df = pd.read_csv(csv_path)
print("Dataset shape:", df.shape)
print("Columns:", df.columns)

# Parameters
max_length = 600
max_tokens = 20000

# Extract texts and labels
texts = df['text'].values
labels = df['source'].values

# Convert labels to numeric values
label_map = {'human': 0, 'ai': 1}
y_data = np.array([label_map[label] for label in labels])

# Define TextVectorization layer
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# Adapt to the text dataset
text_vectorization.adapt(texts)

# Transform text data into tokenized sequences
x_data = text_vectorization(texts).numpy()

# Split data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Check shapes
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

Dataset shape: (5051, 2)
Columns: Index(['text', 'source'], dtype='object')


2025-03-17 15:44:40.949905: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-17 15:44:40.952481: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


x_train shape: (4040, 600)
y_train shape: (4040,)
x_test shape: (1011, 600)
y_test shape: (1011,)


## Zero-Shot Learning
Zero-Shot Learning (ZSL) é uma abordagem onde o modelo é capaz de classificar exemplos de classes que nunca viu durante o treino. Para isso, usam-se embeddings pré-treinados (como BERT, GPT, etc.) e é preciso ajustar o modelo para generalizar para novas classes.

In [3]:
# Zero-Shot Learning Model
def create_zero_shot_model():
    inputs = keras.Input(shape=(max_length,), dtype="int32")
    x = layers.Embedding(max_tokens, 128)(inputs)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    return model

# Compile and train Zero-Shot Model
zero_shot_model = create_zero_shot_model()
zero_shot_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
zero_shot_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/5
127/127 [==============================] - 5s 37ms/step - loss: 0.6654 - accuracy: 0.6010 - val_loss: 0.5733 - val_accuracy: 0.8516
Epoch 2/5
127/127 [==============================] - 4s 33ms/step - loss: 0.3234 - accuracy: 0.9329 - val_loss: 0.1797 - val_accuracy: 0.9535
Epoch 3/5
127/127 [==============================] - 4s 31ms/step - loss: 0.1088 - accuracy: 0.9748 - val_loss: 0.1377 - val_accuracy: 0.9634
Epoch 4/5
127/127 [==============================] - 4s 32ms/step - loss: 0.0575 - accuracy: 0.9894 - val_loss: 0.1185 - val_accuracy: 0.9654
Epoch 5/5
127/127 [==============================] - 4s 32ms/step - loss: 0.0384 - accuracy: 0.9938 - val_loss: 0.1226 - val_accuracy: 0.9575


## One-Shot Learning
One-Shot Learning (OSL) é uma abordagem onde o modelo é treinado para classificar exemplos de classes com base num único exemplo de cada classe. Isto é útil quando há poucos dados disponíveis.

In [4]:
# One-Shot Learning Model
def create_one_shot_model():
    inputs = keras.Input(shape=(max_length,), dtype="int32")
    x = layers.Embedding(max_tokens, 128)(inputs)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    return model

# Compile and train One-Shot Model
one_shot_model = create_one_shot_model()
one_shot_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
one_shot_model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/5
127/127 [==============================] - 4s 33ms/step - loss: 0.6659 - accuracy: 0.6052 - val_loss: 0.5911 - val_accuracy: 0.9327
Epoch 2/5
127/127 [==============================] - 4s 32ms/step - loss: 0.3577 - accuracy: 0.9540 - val_loss: 0.2011 - val_accuracy: 0.9535
Epoch 3/5
127/127 [==============================] - 4s 31ms/step - loss: 0.1225 - accuracy: 0.9725 - val_loss: 0.1318 - val_accuracy: 0.9604
Epoch 4/5
127/127 [==============================] - 4s 32ms/step - loss: 0.0674 - accuracy: 0.9859 - val_loss: 0.1202 - val_accuracy: 0.9614
Epoch 5/5
127/127 [==============================] - 5s 42ms/step - loss: 0.0423 - accuracy: 0.9936 - val_loss: 0.1215 - val_accuracy: 0.9565


## evaluate

In [5]:
# Evaluate models
zero_shot_loss, zero_shot_accuracy = zero_shot_model.evaluate(x_test, y_test)
one_shot_loss, one_shot_accuracy = one_shot_model.evaluate(x_test, y_test)

print("Zero-Shot Model - Loss:", zero_shot_loss, "Accuracy:", zero_shot_accuracy)
print("One-Shot Model - Loss:", one_shot_loss, "Accuracy:", one_shot_accuracy)

32/32 [==============================] - 0s 8ms/step - loss: 0.1215 - accuracy: 0.9565
Zero-Shot Model - Loss: 0.12259894609451294 Accuracy: 0.9574678540229797
One-Shot Model - Loss: 0.1214541643857956 Accuracy: 0.9564787149429321


## Zero-Shot Bart 

In [22]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

# Extrair textos e labels
texts = df['text'].values[:10]  # Selecionar apenas as primeiras 10 entradas para o teste
labels = df['source'].values[:10]  # Labels correspondentes

# Converter labels para valores numéricos
label_map = {'human': 0, 'ai': 1}
y_data = np.array([label_map[label] for label in labels])

# Carregar o pipeline para classificação Zero-Shot com BERT
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Função para classificar com BERT
def classify_with_bart(texts, candidate_labels):
    results = []
    for text in texts:
        result = zero_shot_classifier(text, candidate_labels)
        results.append(result)
    return results

# Lista de rótulos candidatos
candidate_labels = ['human', 'ai']

# Classificar os textos do dataset com o modelo Zero-Shot BERT
classification_results = classify_with_bart(texts, candidate_labels)

# Mostrar as classificações
for i, (text, result) in enumerate(zip(texts, classification_results)):
    print(f"Texto: {text[:150]}...")  # Exibir apenas os primeiros 150 caracteres do texto
    print(f"Classificação: {result['labels'][0]} (Confiança: {result['scores'][0]:.4f})\n")

Texto:   Advanced electromagnetic potentials are indigenous to the classical Maxwell
theory. Generally however they are deemed undesirable and are forcibly
e...
Classificação: ai (Confiança: 0.6850)

Texto: This research paper investigates the question of whether advanced potentials are anomalous. Advanced potentials are a type of psychic phenomenon that ...
Classificação: ai (Confiança: 0.6683)

Texto:   We give an algorithm for finding network encoding and decoding equations for
error-free multicasting networks with multiple sources and sinks. The a...
Classificação: ai (Confiança: 0.7466)

Texto: The paper presents an efficient centralized binary multicast network coding algorithm for any cyclic network. The proposed algorithm aims to optimize ...
Classificação: ai (Confiança: 0.7806)

Texto:   We introduce an exponential random graph model for networks with a fixed
degree distribution and with a tunable degree-degree correlation. We then
i...
Classificação: ai (Confiança: 0.7968)



## Prompt Engineering
O Prompt Engineering é a prática de projetar prompts (entradas) de forma que o modelo gere saídas de alta qualidade. A ideia é fornecer ao modelo o contexto e as instruções mais claras e específicas para obter a resposta desejada.



## RAG (Retrieval-Augmented Generation)
O RAG é uma técnica que combina recuperação de informações com geração de texto. Em vez de confiar apenas na capacidade do modelo de gerar respostas com base no seu treino, o modelo também consulta fontes externas de dados (como uma base de dados ou documentos relevantes) para melhorar a qualidade e a precisão das respostas geradas.

# Predicting for the Competition

In [ ]:
############ NAO IMPLEMENTEI ISTO

from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('best_model_rnn_gru.h5')

print("Model loaded successfully!")

# Carregar o CSV com dados para prever
prediction_csv_path = '../../datasets/dataset1_inputs.csv'
df_predict = pd.read_csv(prediction_csv_path,sep="\t")

# Verificar os dados carregados
print("Prediction dataset shape:", df_predict.shape)
print("Columns:", df_predict.columns)
print("Sample IDs:", df_predict['ID'].head())

# Pré-processar os dados de texto para corresponder ao formato de treinamento
# Converter textos para sequências
sequences = tokenizer.texts_to_sequences(df_predict['Text'].values)

# Padding das sequências para o mesmo tamanho usado no treinamento
x_predict = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

# Fazer previsões com o modelo treinado
predictions = model.predict(x_predict)

# Converter probabilidades para labels binários (0 = Human, 1 = AI)
# Usando 0.5 como threshold - você pode ajustar isso conforme necessário
labels = (predictions > 0.5).astype(int)

# Mapear labels para "AI" e "Human"
label_mapping = {1: "AI", 0: "Human"}
labels_mapped = [label_mapping[label] for label in labels.flatten()]

# Criar um DataFrame com os resultados
results_df = pd.DataFrame({
    'ID': df_predict['ID'],
    'Label': labels_mapped
})

# Exibir uma amostra dos resultados
print("\nAmostra dos resultados de previsão:")
print(results_df.head())

# Salvar em CSV
output_csv_path = 'prediction_results3.csv'
results_df.to_csv(output_csv_path, sep="\t", index=False)
print(f"\nResultados salvos em {output_csv_path}")

Model loaded successfully!
Prediction dataset shape: (30, 2)
Columns: Index(['ID', 'Text'], dtype='object')
Sample IDs: 0    D1-1
1    D1-2
2    D1-3
3    D1-4
4    D1-5
Name: ID, dtype: object
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step

Amostra dos resultados de previsão:
     ID  Label
0  D1-1  Human
1  D1-2  Human
2  D1-3  Human
3  D1-4  Human
4  D1-5     AI

Resultados salvos em prediction_results3.csv
